# Development of driving accident numbers among young drivers with the introduction of BF17
# Final report

## Introduction

## Methods and data pipeline
### Data sources

### The data pipeline

### Loading, transformation and cleaning steps

### Tests

### Occurred problems

## Introduction
In the year 2011 BF17 (Begleitetes Fahren ab 17) was introduced in whole Germany. Young people participating in this program can get a driving license already at the age of 17, one year before the regular permission age, and can drive as long as they are accompanied by an adult companion.\
The benefit of this program was said to be a reduction of driving accidents because young drivers get more expertise together with a experienced companion.

In this project I try to find out if this hope can be confirmed by looking at the number of driving accidents among young drivers in combination with the numbers of BF17 participants in the last years. The question is if a correlation can be found in those two numbers: Has a increasing number of BF17 participants lead to less driving accidents among young people?

## Methods and data pipeline

### Data sources
For this project two open data sources were chosen.

The first question to be solved is if the BF17 program is really used.
This question should be solved by checking the "Fahrerlaubnisse auf Probe" (driving permits on probation), the number of new class B (normal car) driving permits at the age of 17. These can only be obtained if BF17 is used, as the normal minimum age for class B without BF17 is 18.  
These numbers are offered as Excel files from the Kraftfahrbundesamt (KBA) for the years 2017 to 2023.

Then the actual number of accidents is researched. The Statistisches Bundesamt offers the number of traffic accidents of all kinds in their Genesis Destatis data warehouse. This statistic contains all numbers of all age groups and all kinds of accidents so the needed numbers (especially accidents among young drivers but also the number of accidents in general) can be selected. Data is available from the years 2008 to 2023.
TODO: Licenses

### The data pipeline
Because of the two different sources the pipeline consists of two smaller pipelines for both sources:
The kba_pipeline.py for the number of driving permits, and destatis_pipeline.py for the destatis pipeline. By executing the main pipeline with pipeline.sh both pipelines are executed and the data is collected to one single sqlite database.


### Extraction, transformation and cleaning steps
The following steps are done for the both pipelines:

KBA pipeline:
The data does not exist in one single Excel file, but for each year a single Excel file exists. So the following steps are done for each file:
- At first the data is extracted from the KBA website by just downloading the Excel file
- The needed table of the Excel file is extracted with the openpyxl library
- The data is transformed to a pandas dataframe for easier handling
- The data frame is cleaned and transformed by the following steps:
  - All missing numbers (designated in the Excel as - or .) are replaced with proper NaNs
  - The columns are renamed and their data type is changed correctly
  - The year is added as extra column
- Finally the table is appended to the sqlite file table 'Fahrerlaubnisse'

Destatis pipeline:
The data source Genesis is very hard to handle: It suffers from occasional outages and is in general very slow. Also, a (free) account has to created on the Genesis website. Because of this the pipeline has an offline mode which reuses a once downloaded file to transform. The online mode uses the genesis_puller.py file which has the logic to access Genesis.
The following steps are done in this pipeline:
- If the destatis_pipeline.py is started with a Genesis account (Two command line params: Genesis user, Genesis password) => Online mode
  - Retrieve the data from the Genesis data warehouse by doing:
    - Send a batch request to Genesis with the needed table code so that the data is prepared on the Genesis server
    - each 15 seconds: check if the data preparation is done and pull the data if ready; else continue waiting
    - The data preparation can take up to several minutes
  - Write the pulled data into a csv (data/accidents.csv)
- If the destatis_pipeline.py is started without a Genesis account => Offline mode
  - Check if a pre downloaded data/accidents.csv exists
  - If yes: Continue with the transformation with this file
  - If no: Throw an error to the user; end of program
- Load the data from the downloaded/given csv as pandas dataframe
- Then the data is cleaned with the following steps:
  - Unnecessary header lines (because of the DWH format) are removed
  - The DWH slicing is done by removing unnecessary columns and rows (especially vehicle types which are not interesting for this project)
  - The columns are renamed and the data types are changed correctly
  - All missing numbers (designated as - or .) are replaced with proper NaNs
- Finally the dataframe is loaded into the sqlite database as own table

### Tests
